In [ ]:
import os
gpu_device = 2    # número identificador del device puede ser: 0, 1, 2, o 3
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)
from numba import cuda
cuda.select_device(0)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd
import random
import math
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict

In [ ]:
model_name="dmis-lab/biobert-base-cased-v1.1"

In [ ]:
%run ../utils/data.ipynb
%run ../utils/BERT-mix.ipynb

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=5,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.01, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

training_args_all= TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=5,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.00005, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

In [ ]:
len(outputs)

In [ ]:
predicted = np.load("predicted_bio.npy")
tested = np.load("tested_bio.npy")
evaluate_per_question(predicted, tested, DICTIONARY)

In [ ]:
predicted, tested=cross_validation(dataset_final,targets)

In [ ]:
np.save("predicted_bio.npy", predicted)
np.save("tested_bio.npy", tested)

In [ ]:
evaluate_per_question(predicted, tested, DICTIONARY)